In [1]:
###########################################
PROJECT_NAME = 'wheel_axle'
###########################################

In [2]:
import config #place your Openai key here
from openai import OpenAI

# Initialize the OpenAI client with API key
client = OpenAI(api_key=config.OPENAI_API_KEY)

In [3]:
# Global variables to keep track of the conversation
conversation_history = []
last_response = None
model_call_count = 0
# Read the system message once and append to history
with open("system_OpenCASCADE.txt", 'r') as file: # system_OpenCASCADE.txt or system_geo.txt
    system_message = file.read().strip()
conversation_history.append({"role": "system", "content": system_message})

In [4]:
#FUNCTIONS FOR INTERACTING WITH CHATBOT
    
def chat_with_bot(user_messages):
    global last_response
    # Add the new user input to the conversation history
    conversation_history.append({"role": "user", "content": user_messages})
    # Call the OpenAI model
    response = client.chat.completions.create(
        model="gpt-4-1106-preview",
        messages=conversation_history,
        temperature=0
    )
    # Extract the model's response
    model_response = response.choices[0].message.content
    # Update the last response
    last_response = model_response
    # Append the model's response to the conversation history
    conversation_history.append({"role": "assistant", "content": model_response})
    # Print the model's response
    return print(model_response)

#Saves .geo file from response
def extract_and_save_geo_file(response_text): 
    file_path = f"{PROJECT_NAME}.geo"
    try:
        # Use string formatting explained in system_geo.txt 
        start_marker = "//BEGIN_GEO"
        end_marker = f"//END_GEO"

        start_index = response_text.index(start_marker) + len(start_marker)
        end_index = response_text.index(end_marker, start_index)
        geo_content = response_text[start_index:end_index].strip()

        with open(file_path, 'w') as file:
            file.write(geo_content)
        print(f"{file_path} file saved")
        print("Visualize .geo files with GMSH")

    except ValueError as e:
        print(f"ERROR: .geo file NOT SAVED.")

In [5]:
#FUNCTIONS FROM CREATING THE MESH ONCE THE .GEO FILE
import os
import gmsh

def generate_mesh(Mesh_Min_Size, Mesh_Max_Size):
    print("Creating MESH...\n ")
    try:
        current_directory = os.getcwd()
        input_geo_file = os.path.join(current_directory, f'{PROJECT_NAME}.geo') 
        output_msh_file = os.path.join(current_directory, f'{PROJECT_NAME}.msh')  
        # Initialize gmsh
        gmsh.initialize()
        # Open the .geo file
        gmsh.open(input_geo_file)
        gmsh.option.setNumber("Mesh.CharacteristicLengthMin", Mesh_Min_Size)
        gmsh.option.setNumber("Mesh.CharacteristicLengthMax", Mesh_Max_Size)
        # Generate the mesh using default settings
        gmsh.model.mesh.generate(3)
        # Save the mesh to a .msh file
        gmsh.write(output_msh_file)
        # Finalize gmsh
        gmsh.finalize()
        print(f"{PROJECT_NAME}.msh saved")
        print("You can visualize .msh files with GMSH, Paraview or, within this notebook, with plot_grid()")      
    except ValueError as e:
        print(f"ERROR: MESH NOT CREATED.")
        
#(optional)
import pyvista as pv

def plot_grid(output_msh_file):
    # Read mesh file
    mesh = pv.read(output_msh_file)
    # Set the Jupyter backend to 'static'
    pv.set_jupyter_backend('static')
    # Plot the mesh
    plotter = pv.Plotter(notebook=True)
    plotter.add_mesh(mesh, show_edges=True)
    plotter.show()

In [6]:
############ INSTRUCTIONS #######################
# Chat with model with chat_with_bot("..."), 
# when you are satisfied save the .geo file with
# extract_and_save_geo_file(last_response) and
# visualize it with gmsh software.
# Create mesh with generate_mesh() and
# visualize it with gmsh, Paraview, or
# plot_grid(msh_file) for quick visualization
###############################################

In [8]:
chat_with_bot("""Create a wheel and axle. Keep it simple. Assume that both Wheels and axle are cylinders.
The wheels are located at the two ends of the axle. The axis of all cylinders is parallel to the z axis""")

To create a simple wheel and axle system where both the wheels and the axle are cylindrical and their axes are parallel to the z-axis, we need to define the dimensions for the wheels and the axle. Let's assume the following dimensions unless you specify otherwise:

1. Radius of the wheel (Rw): 5 cm
2. Width of the wheel (Ww): 2 cm
3. Radius of the axle (Ra): 1 cm
4. Length of the axle (La): 20 cm

The wheels will be located at the two ends of the axle. The center of the axle will be at the origin (0, 0, 0), and the wheels will be centered at (-La/2, 0, 0) and (La/2, 0, 0) respectively.

Please confirm or provide the dimensions for the wheel radius, wheel width, axle radius, and axle length. Once I have the dimensions, I can proceed to create the .geo file.


In [9]:
chat_with_bot("Yes, go ahead")

Based on the provided assumptions, I will create a .geo file for a wheel and axle system with the following dimensions:

- Radius of the wheel (Rw): 5 cm
- Width of the wheel (Ww): 2 cm
- Radius of the axle (Ra): 1 cm
- Length of the axle (La): 20 cm

Here is the .geo file content:

```
//BEGIN_GEO
// Switch to OpenCASCADE geometry kernel
SetFactory("OpenCASCADE");

// Define the dimensions
Rw = 5; // Radius of the wheel in cm
Ww = 2; // Width of the wheel in cm
Ra = 1; // Radius of the axle in cm
La = 20; // Length of the axle in cm

// Define the axle cylinder
Cylinder(1) = {0, 0, -La/2, 0, 0, La, Ra};

// Define the left wheel cylinder
Cylinder(2) = {-La/2 - Ww, 0, 0, 0, 0, Ww, Rw};

// Define the right wheel cylinder
Cylinder(3) = {La/2, 0, 0, 0, 0, Ww, Rw};

// Define physical groups for the axle and wheels
Physical Volume("Axle") = {1};
Physical Volume("LeftWheel") = {2};
Physical Volume("RightWheel") = {3};
//END_GEO
```

This file defines a simple wheel and axle system with two

In [11]:
extract_and_save_geo_file(last_response)

wheel_axle.geo file saved
Visualize .geo files with GMSH


In [16]:
chat_with_bot("""The wheels are not correctly located. The should be located at
the ends of the axle and on the same z axis: wheel|axle|wheel """)

I apologize for the oversight. Let's position the wheels correctly at the ends of the axle, ensuring they are on the same z-axis in the configuration: wheel|axle|wheel. The wheels should be positioned such that the axle is sandwiched between the two wheels.

Here's the revised .geo file content:

```
//BEGIN_GEO
// Switch to OpenCASCADE geometry kernel
SetFactory("OpenCASCADE");

// Define the dimensions
Rw = 5; // Radius of the wheel in cm
Ww = 2; // Width of the wheel in cm
Ra = 1; // Radius of the axle in cm
La = 20; // Length of the axle in cm

// Define the axle cylinder
// The axle is centered at the origin and spans from -La/2 to La/2 along the z-axis
Cylinder(1) = {0, 0, -La/2, 0, 0, La, Ra};

// Define the left wheel cylinder
// The left wheel is positioned such that its inner face is flush with the left end of the axle
Cylinder(2) = {0, 0, -La/2 - Ww, 0, 0, Ww, Rw};

// Define the right wheel cylinder
// The right wheel is positioned such that its inner face is flush with the

In [24]:
extract_and_save_geo_file(last_response)

wheel_axle.geo file saved
Visualize .geo files with GMSH


In [26]:
generate_mesh(Mesh_Min_Size = 0.1, Mesh_Max_Size = 0.5)

Creating MESH...
 
wheel_axle.msh saved
You can visualize .msh files with GMSH, Paraview or, within this notebook, with plot_grid()
